# EDA дипломного проекта

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import numpy as np

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
plt.rcParams.update({'font.size': 20})

TRESHOLD_CORR = 0.01

## Общая информация о датасете

In [ ]:
DATA_PATH = "../input/unsw-nb15/"

### Сводная таблица по всем признакам

|     Имя признака | Тип признака | Описание признака                                            |
| ---------------: | -----------: | ------------------------------------------------------------ |
|            srcip |      nominal | ip-адрес отправителя                                         |
|            sport |      integer | порт отправителя                                             |
|            dstip |      nominal | ip-адрес получателя                                          |
|           dsport |      integer | порт получателя                                              |
|            proto |      nominal | тип протокола, вложенного в ip (TCP, UDP, ICMP, ESP, ...)    |
|            state |      nominal | флаг состояния протокола транспорного уровня (характерен для протокола TCP) |
|              dur |        Float | длительность сессии                                          |
|           sbytes |      Integer | размер сессии (байт) от отправителя к получателю             |
|           dbytes |      Integer | размер сессии (байт) от получателя к отправителю             |
|             sttl |      Integer | значение параметра TTL протокола TCP в направлении от инициатору к ответчику |
|             dttl |      Integer | значение параметра TTL протокола TCP в обратном направлении  |
|            sloss |      Integer | количество потерянных или повторно переданных пакетов от инициатора к ответчику |
|            dloss |      Integer | количество потерянных или повторно переданных пакетов от ответчика к получателю |
|          service |      nominal | тип службы (http, ftp, smtp, ssh, dns, ftp-data ,irc и т.п.) |
|            Sload |        Float | скорость в прямом направлении (бит/сек)                      |
|            Dload |        Float | скорость в обратном направлении (бит/сек)                    |
|            Spkts |      integer | количество пакетов в прямом направлении                      |
|            Dpkts |      integer | количество пакетов в обратном направлении                    |
|             swin |      integer | размер окна в прямом направлении (параметр протокола TCP)    |
|             dwin |      integer | размер окна в обратном направлении (параметр протокола TCP)  |
|            stcpb |      integer | стартовое значение счётчика пакетов в прямом направлении (параметр протокола TCP ) |
|            dtcpb |      integer | стартовое значение счётчика пакетов в обратном направлении (параметр протокола TCP ) |
|          smeansz |      integer | средний размер пакета в прямом направлении                   |
|          dmeansz |      integer | средний размер пакета в обратном направлении                 |
|      trans_depth |      integer | размер вложенности html запроса                              |
|      res_bdy_len |      integer | реальный размер несжатых данных сессии                       |
|             Sjit |        Float | временной разброс между пакетами в прямом направлении (mSec) |
|             Djit |        Float | временной разброс между пакетами в обратном направлении (mSec) |
|            Stime |    Timestamp | временная метка начала сессии                                |
|            Ltime |    Timestamp | временная метка окончания сессии                             |
|          Sintpkt |        Float | интервал следования пакетов в прмом направлении (mSec)       |
|          Dintpkt |        Float | интервал следования пакетов в обратном направлении (mSec)    |
|           tcprtt |        Float | полное время жизни сеиссии                                   |
|           synack |        Float | время установления TCP-сессии (между пакетом SYN и SYN_ACK)  |
|           ackdat |        Float | время установления TCP-сессии (между пакетом SYN_ACK и ACK)  |
|  is_sm_ips_ports |       Binary | флаг, указывающий на то, что ip-адреса отправителя и получателя и соответствующие порты сессий совпадают |
|     ct_state_ttl |      Integer | характерный для конкретного state параметр ttl               |
| ct_flw_http_mthd |      Integer | количество потоков, открытых с помощью методов GET/POST      |
|     is_ftp_login |       Binary | содержит ли ftp-соединение сеанс аутентификации пользователя |
|       ct_ftp_cmd |      integer | количество команд ftp-сессии                                 |
|       ct_srv_src |      integer | количество соединений к такойже службе по тому же адресу отправителя на 100 соединенийй за последнее время |
|       ct_srv_dst |      integer | количество соединений к такойже службе по тому же адресу получателя на 100 соединений за последнее время |
|       ct_dst_ltm |      integer | No. of connections of the same source address (1) in 100 connections according to the last time (26). |
|      ct_src_ ltm |      integer | No. of connections of the same source address (1) in 100 connections according to the last time (26). |
| ct_src_dport_ltm |      integer | No of connections of the same source address (1) and the destination port (4) in 100 connections according to the last time (26). |
| ct_dst_sport_ltm |      integer | No of connections of the same destination address (3) and the source port (2) in 100 connections according to the last time (26). |
|   ct_dst_src_ltm |      integer | No of connections of the same source (1) and the destination (3) address in in 100 connections according to the last time (26). |
|       attack_cat |      nominal | тип атаки                                                    |
|            Label |       binary | тип сессии (0 - нормальный трафик, 1 - сетвая атака)         |

### Общая информация о данных

In [ ]:
train_set =  pd.read_csv(DATA_PATH + 'UNSW_NB15_testing-set.csv')
train_set.drop(['id'], axis=1, inplace=True)
train_set.head()

In [ ]:
train_set.shape

In [ ]:
test_set =  pd.read_csv(DATA_PATH + 'UNSW_NB15_training-set.csv')
test_set.drop(['id'], axis=1, inplace=True)
test_set.head()

In [ ]:
print(
    f'Обучающая выборка содержит {train_set.shape[0]} элементов. \n'
    f'Тестовая выборка содержит {test_set.shape[0]} элементов.\n'
    f'Общий размер выборки {test_set.shape[0]+train_set.shape[0]} элементов.\n'
    f'Cоотношение размера тестовой выборки к общему объёму выборки составляет {test_set.shape[0]/(test_set.shape[0]+train_set.shape[0]):.2f}%.'
)

In [ ]:
train_set['sample'] = 1 # помечаем где у нас трейн
test_set['sample'] = 0 # помечаем где у нас тест

data = test_set.append(train_set, sort=False).reset_index(drop=True) # объединяем
print (train_set.shape, test_set.shape, data.shape)

### Подробный анализ признаков

In [ ]:
def feature_info(column, bins=0, normalize=True, values=False, threshold=10):
    '''
    Функция выводит статистическую информацию о признаке
     :param column: Столбец значений соответствующего признака
     :param bins: параметр для вывода распределения значений признака (на сколько частей разбить диапазон 
                  возможных значений) 
     :param normalization: нормализовать или нет признак
     :param values: Выводить или нет список значений признака
     :param threshold: порог для выводаа наиболее часто встресающихся значений (например 10)
     :reurn: итоговую модель
    '''

    if (bins == 0):
        bins = len(column.value_counts(dropna=True, normalize=normalize))

    print(f"Тип признака: {column.dtype}\nЗначения признака:")

    if column.dtype != 'object':
        column.hist(bins=bins, align='left')
        display(column.describe())
        if values == True:
            display(
                pd.DataFrame(
                    column.value_counts(dropna=False,
                                        normalize=normalize).round(3)))
    else:
        display(
            pd.DataFrame(
                column.value_counts(dropna=False,
                                    normalize=normalize).round(3)))

    print(
        f"Различных значений c учётом NaN: {len(column.value_counts(dropna=False, normalize=normalize))}\n\
              \t  без учёта NaN: {len(column.value_counts(dropna=True, normalize=normalize))}\
        \nПропусков: {column.isnull().sum()}")
    print(
        # Число 10 взято для ориентира, можно брать другое
        f"Значений, встретившихся в столбце более {threshold} раз:",
        (column.value_counts() > threshold).sum())

In [ ]:
# Функция для поиска выбрпосов методом IQR
def IQR_stat(column, senseble_rate=0):
    IQR = column.quantile(0.75) - column.quantile(0.25)
    perc25 = column.quantile(0.25)
    perc75 = column.quantile(0.75)
    print(
        '25-й перцентиль: {},\n'.format(perc25),
        '75-й перцентиль: {},\n'.format(perc75), "IQR: {}, \n".format(IQR),
        "Границы выбросов: [{f}, {l}].\n".format(f=perc25 - 1.5 * IQR,
                                                 l=perc75 + 1.5 * IQR))
    column.loc[column.between(perc25 - 1.5 * IQR,
                              perc75 + 1.5 * IQR)].hist(bins=20,
                                                        range=(0, 100),
                                                        label='IQR')
    column.loc[column <= senseble_rate].hist(alpha=0.5,
                                             bins=20,
                                             range=(0, 100),
                                             label='Здравый смысл')
    plt.legend()

In [ ]:
numerical_features = [] # Числовые признаки, требующие нормировки
categorical_features = [] # Категориальные признаки, требующие перевода в числовой формат
MAX_BINS = 20


#### dur - Длительность сессии

In [ ]:
feature_info(data.dur, bins=MAX_BINS)

In [ ]:
data[data.dur>20].label.value_counts()

In [ ]:
data[((data.dur>20) & (data.label==0))].proto.value_counts()

In [ ]:
data[((data.dur>20) & (data.label==1))].attack_cat.value_counts()

- Выбросоы объясняются особенностями регистрации сетевых протоколов ARP и т.п

In [ ]:
numerical_features.append('dur')

#### proto тип протокола (поле в протоколе ip)

In [ ]:
 feature_info(data.proto, normalize=True)

In [ ]:
data.proto = data.proto.apply(lambda x: x if x in ['tcp','udp', 'unas'] else 'other')

In [ ]:
feature_info(data.proto, normalize=True)

In [ ]:
categorical_features.append('proto')

#### service - служба (ftp, http ...)

In [ ]:
 feature_info(data.service)

In [ ]:
data.service = data.service.apply(lambda x: x if x in ['-','dns', 'http'] else 'other')

In [ ]:
feature_info(data.service)

In [ ]:
categorical_features.append('service')

In [ ]:
categorical_features

#### state - состояние протокола

In [ ]:
feature_info(data['state'])

In [ ]:
data.state = data.state.apply(lambda x: x if x in ['FIN','INT', 'CON'] else 'other')

In [ ]:
feature_info(data.state)

In [ ]:
categorical_features.append('state')

#### spkts - отправлено пакетов от инициатора к получателю

In [ ]:
feature_info(data['spkts'], MAX_BINS*5)

In [ ]:
data[((data.spkts>200) & (data.label==1))]   #.proto.value_counts()

In [ ]:
numerical_features.append('spkts')

#### dpkts - количесвто пакетов от получателя к инициатору

In [ ]:
feature_info(data['dpkts'], bins=MAX_BINS*5)

In [ ]:
numerical_features.append('dpkts')

#### sbytes - передано байт от инициатора к ответчику

In [ ]:
feature_info(data['sbytes'], bins=MAX_BINS)

In [ ]:
numerical_features.append('sbytes')

#### dbytes - передано байт от ответчика к инициатору

In [ ]:
feature_info(data['dbytes'], bins=MAX_BINS)

In [ ]:
numerical_features.append('dbytes')

#### rate - соотношение переданных/полученных

In [ ]:
feature_info(data['rate'], bins=MAX_BINS)

In [ ]:
numerical_features.append('rate')

#### sttl - параметр TTL инициатора

In [ ]:
feature_info(data['sttl'])

In [ ]:
data.sttl.value_counts()

In [ ]:
data.sttl.astype('str')

In [ ]:
numerical_features.append('sttl')

#### dttl - параметр TTL ответчика

In [ ]:
feature_info(data['dttl'])

In [ ]:
data.dttl.value_counts()

In [ ]:
data.dttl.astype('str')

In [ ]:
numerical_features.append('sttl')

#### sload - скорость от инициатора к ответчику

In [ ]:
feature_info(data['sload'], bins=20)

In [ ]:
numerical_features.append('sload')

#### dload - скорость от инициатора к ответчику

In [ ]:
feature_info(data['dload'], bins=20)

In [ ]:
numerical_features.append('dload')

#### sloss - количество потерянных или повторно переданных пакетов от инициатора к ответчику

In [ ]:
feature_info(data['sloss'], bins=20)

In [ ]:
numerical_features.append('sloss')

#### dloss - количество потерянных или повторно переданных пакетов от ответчика  к инициатору

In [ ]:
feature_info(data['dloss'], bins=20)

In [ ]:
numerical_features.append('dloss')

#### sinpkt - средний интервал следования пакетов от инициатора к ответчику

In [ ]:
feature_info(data['sinpkt'], bins=100)

In [ ]:
numerical_features.append('sinpkt')

#### dinpkt - средний интервал следования пакетов от ответчика  к инициатору

In [ ]:
feature_info(data['dinpkt'], bins=100)

In [ ]:
numerical_features.append('dinpkt')

#### sjit - отклонение от среднего интервала  пакетов от инициатора  к ответчику

In [ ]:
feature_info(data['sjit'], bins=10)

In [ ]:
numerical_features.append('sjit')

#### djit - отклонение от среднего интервала  пакетов от ответчика  к инициатору 

In [ ]:
feature_info(data['djit'], bins=100)

In [ ]:
numerical_features.append('djit')

#### swin - размер окна сессии от  инициатору к ответчику

In [ ]:
feature_info(data['swin'])  #, bins=10)

In [ ]:
data.swin.value_counts()

In [ ]:
numerical_features.append('swin')

#### stcpb - стартовый номер счётчика пакетов от инициатора к ответчику

In [ ]:
feature_info(data['stcpb'], bins=100)

In [ ]:
numerical_features.append('stcpb')

#### dtcpb - стартовый номер счётчика пакетов от инициатора к ответчику

In [ ]:
feature_info(data['dtcpb'], bins=100)

In [ ]:
numerical_features.append('dtcpb')

#### dwin - размер окна сессии от  ответчика к инициатору

In [ ]:
feature_info(data['dwin'])  #, bins=10)

In [ ]:
data.dwin.value_counts()

In [ ]:
numerical_features.append('dwin')

#### tcprtt - время установления соединения

In [ ]:
feature_info(data['tcprtt'], bins=MAX_BINS)

In [ ]:
numerical_features.append('tcprtt')

#### synack - время между пакетами syn и synack

In [ ]:
feature_info(data['synack'], bins=MAX_BINS)

In [ ]:
numerical_features.append('synack')

#### ackdat - время между пакетами synack и ack

In [ ]:
feature_info(data['ackdat'], bins=MAX_BINS)

In [ ]:
numerical_features.append('ackdat')

#### smean - среднее размер переданных пакетов от инициатора к ответчику

In [ ]:
feature_info(data['smean'], bins=MAX_BINS)

In [ ]:
numerical_features.append('smean')

#### dmean - среднее размер переданных пакетов от инициатора к ответчику

In [ ]:
feature_info(data['dmean'], bins=MAX_BINS)

In [ ]:
numerical_features.append('dmean')

#### trans_depth - глубина вложенности запросов

In [ ]:
feature_info(data['trans_depth'], bins=MAX_BINS)

In [ ]:
data['trans_depth'].value_counts()

In [ ]:
numerical_features.append('trans_depth')

#### response_body_len - размер информационной нагрузки от ответчика к инициатору

In [ ]:
feature_info(data['response_body_len'], bins=MAX_BINS)

In [ ]:
numerical_features.append('response_body_len')

#### ct_srv_src - количество соединений с соответствующей службой и адресом инициатора

In [ ]:
feature_info(data['ct_srv_src'], bins=MAX_BINS)

In [ ]:
numerical_features.append('ct_srv_src')

#### ct_state_ttl - размер диапазона значений ttl для соответсвующего state

In [ ]:
feature_info(data['ct_state_ttl'], bins=MAX_BINS)

In [ ]:
data['ct_state_ttl'].value_counts()

In [ ]:
numerical_features.append('ct_state_ttl')

#### ct_dst_ltm - количество соединений от соответствующего ip-dst

In [ ]:
feature_info(data['ct_dst_ltm'], bins=MAX_BINS)

In [ ]:
numerical_features.append('ct_dst_ltm')

#### ct_src_dport_ltm - количество соединений от соответствующего ip-src на port_dst

In [ ]:
feature_info(data['ct_src_dport_ltm'], bins=MAX_BINS)

In [ ]:
numerical_features.append('ct_src_dport_ltm')

#### ct_dst_sport_ltm - количество соединений от соответствующего dst-port на port_src

In [ ]:
feature_info(data['ct_dst_sport_ltm'], bins=MAX_BINS)

In [ ]:
numerical_features.append('ct_dst_sport_ltm')

#### ct_dst_src_ltm - количество соединений от соответствующего dst-port на адрес src

In [ ]:
feature_info(data['ct_dst_src_ltm'], bins=MAX_BINS)

In [ ]:
numerical_features.append('ct_dst_src_ltm')

#### is_ftp_login - сеанс авторизация ftp?

In [ ]:
data['is_ftp_login'].value_counts()

- Признак содержит выброс (в бинарном признаке не может быть 2, 4), заменяется на 0

In [ ]:
data.is_ftp_login = data.is_ftp_login.apply(lambda x: 1 if x==2 or x==4 else x)

#### ct_ftp_cmd - количество потоков имеющих управляющие команды

In [ ]:
data['ct_ftp_cmd'].value_counts()

In [ ]:
numerical_features.append('ct_ftp_cmd')

#### ct_flw_http_mthd - количество потоков содержащих методы POST/GET для протокола http

In [ ]:
data['ct_flw_http_mthd'].value_counts()

In [ ]:
numerical_features.append('ct_flw_http_mthd')

#### ct_src_ltm - количесвто соедиений от соответствующего ip-источника

In [ ]:
feature_info(data['ct_src_ltm'])

In [ ]:
numerical_features.append('ct_src_ltm')

#### ct_srv_dst - с заданной службой и адресом ответчика

In [ ]:
feature_info(data['ct_srv_dst'])

In [ ]:
numerical_features.append('ct_srv_dst')

#### attack_cat - тип атаки

In [ ]:
feature_info(data['attack_cat'])

- Целевые классы распределены не равномерно

In [ ]:
encoder = LabelEncoder()
data.attack_cat = encoder.fit_transform(data.attack_cat)


In [ ]:
param = encoder.inverse_transform([0,1,2,3,4,5,6,7,8,9])

In [ ]:
param

#### label - вредоносный трафик или нет

In [ ]:
data['label'].value_counts()

In [ ]:
data.corr()['label'].sort_values(ascending=False)

In [ ]:
for col in data.columns:
    print(col, len(data[col].unique()))

### Преобразование категриальных и нормировка числовых признаков

In [ ]:
scaler = MinMaxScaler()
for feature in numerical_features:
    print(feature, data[feature].max())
    data[feature] = np.log(data[feature].max()+100 -data[feature])
    data[feature] = scaler.fit_transform(data[[feature]])[:,0]


In [ ]:
for column in categorical_features:
        data[column] = data[column].astype('category').cat.codes

data = pd.get_dummies(data, columns=categorical_features, dummy_na=False)

In [ ]:
data.head()

#### Наиболее скоррелированные с целевыми переменными признаки

In [ ]:
corr_matrix = data.corr()

In [ ]:
label_correlated = corr_matrix[np.abs(
    corr_matrix['label'] > TRESHOLD_CORR)]['label'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(label_correlated.index, label_correlated.values);
plt.xticks(rotation=90);

In [ ]:
fig, ax = plt.subplots(figsize=(30,30)) 

sns.heatmap(corr_matrix[label_correlated.index].loc[label_correlated.index], square=True,
            annot=True, fmt=".2f", linewidths=0.1, cmap="RdBu", ax=ax)

In [ ]:
attack_correlated = corr_matrix[np.abs(
    corr_matrix['attack_cat'] > TRESHOLD_CORR)]['attack_cat'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(attack_correlated.index, attack_correlated.values);
plt.xticks(rotation=90);

In [ ]:
fig, ax = plt.subplots(figsize=(30,30)) 
sns.heatmap(corr_matrix[attack_correlated.index].loc[attack_correlated.index], square=True,
            annot=True, fmt=".2f", linewidths=0.1,  ax=ax)

In [ ]:
malignant_data = data[data.label==1]

In [ ]:
malligant_corr = malignant_data.corr()

In [ ]:
malligan_correlated = malligant_corr[np.abs(
    malligant_corr['attack_cat'] > TRESHOLD_CORR)]['attack_cat'].sort_values(ascending=False)

In [ ]:
maligant_index = malligan_correlated.index.tolist()

In [ ]:
maligant_index

In [ ]:
def print_corelated_features(matrix, valueable_fetures_list, corr_treshold = 0.7):
    """
    Функция выводит список наиболее скоррелированных признаков.
    :param matrix - корреляционная матрица признаков
    :param valueable_fetures_list - список наиболее скоррелированных признаков
    :corr_treshold - порог значимости корреляции
    """
    for index in valueable_fetures_list:
        for column in valueable_fetures_list:
            if np.abs(matrix[index][column])>0.7 and index!=column:
                print (index, column, matrix[index][column])

In [ ]:
for index in maligant_index:
        for column in maligant_index:
            if np.abs(malligant_corr[index][column])>0.7 and index!=column:
                maligant_index.remove(column)

In [ ]:
maligant_index.append('sample')

In [ ]:
maligant_index

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
data_2d = pca.fit_transform(StandardScaler().fit_transform(
    data.select_dtypes(exclude='object').drop(columns= ['dtcpb', 'stcpb'])))
mask = data.label==1

In [ ]:
plt.scatter(data_2d[mask,0], data_2d[mask,1], label = 'attack data')
plt.scatter(data_2d[~mask,0], data_2d[~mask,1], label = 'normal data')
plt.title('2-D PCA visualization of UNSW NB-15 training dataset')
plt.legend();